In [48]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
pip install pypdf2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import gradio as gr
import openai
import os
from pathlib import Path
import difflib
import json
from typing import Tuple, List, Dict
import docx
import PyPDF2
import io
import re
from datetime import datetime

In [52]:
def extract_text_from_file(self, file_path: str) -> str:  # Added proper indentation
        """Extract text from various file formats"""
        file_extension = Path(file_path).suffix.lower()
        
        try:
            if file_extension == '.txt':
                with open(file_path, 'r', encoding='utf-8') as file:
                    return file.read()
            
            elif file_extension == '.docx':
                doc = docx.Document(file_path)
                text = []
                for paragraph in doc.paragraphs:
                    text.append(paragraph.text)
                return '\n'.join(text)
            
            elif file_extension == '.pdf':
                text = []
                with open(file_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page in pdf_reader.pages:
                        text.append(page.extract_text())
                return '\n'.join(text)
            
            else:
                return f"Unsupported file format: {file_extension}"
        
        except Exception as e:
            return f"Error reading file: {str(e)}"

In [53]:
class DocumentComparator:
    def _init_(self):  # Fixed: double underscores
        self.supported_formats = ['.txt', '.docx', '.pdf']
    
    def extract_text_from_file(self, file_path: str) -> str:
        """Extract text from various file formats"""
        file_extension = Path(file_path).suffix.lower()
        
        try:
            if file_extension == '.txt':
                with open(file_path, 'r', encoding='utf-8') as file:
                    return file.read()
            
            elif file_extension == '.docx':
                doc = docx.Document(file_path)
                text = []
                for paragraph in doc.paragraphs:
                    text.append(paragraph.text)
                return '\n'.join(text)
            
            elif file_extension == '.pdf':
                text = []
                with open(file_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page in pdf_reader.pages:
                        text.append(page.extract_text())
                return '\n'.join(text)
            
            else:
                return f"Unsupported file format: {file_extension}"
        
        except Exception as e:
            return f"Error reading file: {str(e)}"
    
    def extract_text_from_uploaded_file(self, file) -> str:
        """Extract text from Gradio uploaded file"""
        if file is None:
            return ""
        
        file_path = file.name
        return self.extract_text_from_file(file_path)
    
    def generate_diff_html(self, text1: str, text2: str, file1_name: str = "Document 1", file2_name: str = "Document 2") -> str:
        """Generate HTML diff highlighting changes between two texts"""
        
        # Split texts into lines for better comparison
        lines1 = text1.splitlines(keepends=True)
        lines2 = text2.splitlines(keepends=True)
        
        # Generate unified diff
        diff = list(difflib.unified_diff(
            lines1, lines2, 
            fromfile=file1_name, 
            tofile=file2_name, 
            lineterm='', 
            n=3
        ))
        
        # Generate HTML diff
        html_diff = difflib.HtmlDiff(wrapcolumn=100)
        diff_html = html_diff.make_file(
            lines1, lines2, 
            fromdesc=file1_name, 
            todesc=file2_name,
            context=True,
            numlines=2
        )
        
        return diff_html
    
    def get_text_differences(self, text1: str, text2: str) -> Dict:
        """Get structured differences between two texts"""
        
        lines1 = text1.splitlines()
        lines2 = text2.splitlines()
        
        differ = difflib.Differ()
        diff = list(differ.compare(lines1, lines2))
        
        changes = {
            'additions': [],
            'deletions': [],
            'modifications': [],
            'stats': {}
        }
        
        for line in diff:
            if line.startswith('+ '):
                changes['additions'].append(line[2:])
            elif line.startswith('- '):
                changes['deletions'].append(line[2:])
        
        changes['stats'] = {
            'additions_count': len(changes['additions']),
            'deletions_count': len(changes['deletions']),
            'total_changes': len(changes['additions']) + len(changes['deletions'])
        }
        
        return changes
    
    def summarize_differences_with_openai(self, differences: Dict, text1: str, text2: str) -> str:
        """Use OpenAI to generate a summary of differences"""
        
        try:
            # Set your OpenAI API key here
            openai.api_key = "your-api-key-here"  # Replace with your actual API key
            
            # Prepare prompt for OpenAI
            prompt = f"""
            Analyze the differences between two documents and provide a comprehensive summary.
            
            Document Statistics:
            - Additions: {differences['stats']['additions_count']} lines
            - Deletions: {differences['stats']['deletions_count']} lines
            - Total Changes: {differences['stats']['total_changes']} lines
            
            Added Content (first 10 lines):
            {chr(10).join(differences['additions'][:10])}
            
            Deleted Content (first 10 lines):
            {chr(10).join(differences['deletions'][:10])}
            
            Please provide:
            1. A brief summary of the main changes
            2. Key differences in content or structure
            3. Potential impact of these changes
            4. Any patterns you notice in the modifications
            
            Keep the summary concise but informative.
            """
            
            # Make API call to OpenAI (updated for newer API)
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a document analysis expert. Provide clear, concise summaries of document changes."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=500,
                temperature=0.3
            )
            
            return response.choices[0].message.content.strip()
        
        except Exception as e:
            return f"Error generating AI summary: {str(e)}. Please check your OpenAI API key and connection."
    
    def compare_documents(self, file1, file2) -> Tuple[str, str, str]:
        """Main function to compare two documents"""
        
        if file1 is None or file2 is None:
            return "Please upload both documents to compare.", "", ""
        
        # Extract text from both files
        text1 = self.extract_text_from_uploaded_file(file1)
        text2 = self.extract_text_from_uploaded_file(file2)
        
        if "Error reading file" in text1 or "Error reading file" in text2:
            return f"Error reading files:\nFile 1: {text1}\nFile 2: {text2}", "", ""
        
        # Get differences
        differences = self.get_text_differences(text1, text2)
        
        # Generate HTML diff
        html_diff = self.generate_diff_html(text1, text2, 
                                           getattr(file1, 'name', 'Document 1'),
                                           getattr(file2, 'name', 'Document 2'))
        
        # Create summary
        summary = f"""
        ## 📊 Document Comparison Summary
        
        *Statistics:*
        - Lines Added: {differences['stats']['additions_count']}
        - Lines Deleted: {differences['stats']['deletions_count']}
        - Total Changes: {differences['stats']['total_changes']}
        
        *Analysis:*
        {self.summarize_differences_with_openai(differences, text1, text2)}
        """
        
        return summary, html_diff, "Comparison completed successfully!"

In [54]:
def create_gradio_interface():
    """Create and return the Gradio interface"""
    
    # Create an instance of the DocumentComparator
    comparator = DocumentComparator()
    
    # Create the Gradio interface
    with gr.Blocks(title="📄 Document Comparator", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 📄 Document Comparator")
        gr.Markdown("Upload two documents to compare their differences. Supports TXT, DOCX, and PDF files.")
        
        with gr.Row():
            with gr.Column():
                file1 = gr.File(
                    label="📄 Document 1", 
                    file_types=['.txt', '.docx', '.pdf']
                )
            with gr.Column():
                file2 = gr.File(
                    label="📄 Document 2", 
                    file_types=['.txt', '.docx', '.pdf']
                )
        
        compare_btn = gr.Button("🔍 Compare Documents", variant="primary", size="lg")
        
        with gr.Row():
            with gr.Column():
                summary_output = gr.Markdown(label="📊 Summary")
            
        with gr.Row():
            diff_output = gr.HTML(label="📋 Detailed Differences")
        
        status_output = gr.Textbox(label="Status", visible=False)
        
        # Set up the comparison function
        compare_btn.click(
            fn=comparator.compare_documents,
            inputs=[file1, file2],
            outputs=[summary_output, diff_output, status_output]
        )
    
    return demo

In [55]:
# Create and launch the interface with explicit output
print("Starting Document Comparator...")
demo = create_gradio_interface()
print("Interface created successfully!")

# Launch with more verbose settings
demo.launch(
    share=False,
    server_name="127.0.0.1",
    server_port=7860,
    show_error=True,
    inbrowser=True,
    debug=True,  # Added for more verbose output
    quiet=False  # Ensure output is shown
)

Starting Document Comparator...
Interface created successfully!
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
# Check if the server is already running
import requests
try:
    response = requests.get("http://127.0.0.1:7860")
    if response.status_code == 200:
        print("✅ Server is running at: http://127.0.0.1:7860")
        print("Open this URL in your browser!")
    else:
        print("❌ Server not responding")
except:
    print("❌ Server not running")

❌ Server not running
